In [30]:
import pandas as pd
import numpy as np
import datetime
import os 

In [31]:
#read in the latest dataset
time_series = pd.read_csv("../../../databases/JHU/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,149,149,151,151,151,151,152,152,152,154
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,15,15,15,16,16,19,19,19,19,19
3,630,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1968,2031,2156,2173,2198,2256,2299,2329,2427,2542
4,850,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,66,66,68,68,69,69,69,69,69,69


In [32]:
#keep only data from NC counties
time_series = time_series[time_series["Province_State"] == "North Carolina"]
time_series = time_series[time_series["Admin2"] != "Out of NC"]
time_series = time_series[time_series["Admin2"] != "Unassigned"]
time_series.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
1895,84037001,US,USA,840,37001.0,Alamance,North Carolina,US,36.043470,-79.399761,...,128,143,149,152,173,183,183,190,196,196
1896,84037003,US,USA,840,37003.0,Alexander,North Carolina,US,35.922380,-81.177519,...,6,8,8,10,12,12,13,14,15,15
1897,84037005,US,USA,840,37005.0,Alleghany,North Carolina,US,36.493609,-81.128570,...,7,7,7,7,8,8,8,10,10,10
1898,84037007,US,USA,840,37007.0,Anson,North Carolina,US,34.974032,-80.099533,...,31,33,37,38,38,38,39,40,41,43
1899,84037009,US,USA,840,37009.0,Ashe,North Carolina,US,36.432962,-81.498627,...,5,6,7,8,13,13,14,16,17,17


In [33]:
#define a dataframe with just the county information
nc_counties = time_series[["FIPS", "Admin2","Province_State","Lat","Long_"]]
nc_counties.head()

,FIPS,Admin2,Province_State,Lat,Long_
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570
1898,37007.0,Anson,North Carolina,34.974032,-80.099533
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627


In [34]:
#define variable for latest day of data to be pulled
startdate = datetime.datetime(2020,3,4)
today = datetime.datetime.today()
offset = 53 + (today-startdate).days
offset

126

In [35]:
#create a dataframe that is just the number of total cases by county for each day
dates = time_series.iloc[:,53:offset]
dates.head()

,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,...,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20
1895,0,0,0,0,0,0,0,0,0,0,...,128,143,149,152,173,183,183,190,196,196
1896,0,0,0,0,0,0,0,0,0,0,...,6,8,8,10,12,12,13,14,15,15
1897,0,0,0,0,0,0,0,0,0,0,...,7,7,7,7,8,8,8,10,10,10
1898,0,0,0,0,0,0,0,0,0,0,...,31,33,37,38,38,38,39,40,41,43
1899,0,0,0,0,0,0,0,0,0,0,...,5,6,7,8,13,13,14,16,17,17


In [36]:
#create an empty dataframe that will be filled with data needed to calculate new cases and 14-day rolling average
nc_time_series = pd.DataFrame(columns = ["FIPS","Admin2","Province_State","Lat","Long_", "Confirmed",
                                             "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])

In [37]:
#pull the values to fill the cells for calculating new cases and 14-day rolling average
cols = dates.columns

for i in range(14,len(cols)):
    one_day = pd.DataFrame(columns = ["Confirmed", "Yesterday", "Day_3","Day_4","Day_5","Day_6","Day_7",
                                             "Day_8","Day_9","Day_10","Day_11","Day_12","Day_13","Day_14","Date"])
    one_day["Confirmed"] = dates.iloc[:,i]
    one_day["Yesterday"] = dates.iloc[:,i-1]
    one_day["Day_3"] = dates.iloc[:,i-2]
    one_day["Day_4"] = dates.iloc[:,i-3]
    one_day["Day_5"] = dates.iloc[:,i-4]
    one_day["Day_6"] = dates.iloc[:,i-5]
    one_day["Day_7"] = dates.iloc[:,i-6]
    one_day["Day_8"] = dates.iloc[:,i-7]
    one_day["Day_9"] = dates.iloc[:,i-8]
    one_day["Day_10"] = dates.iloc[:,i-9]
    one_day["Day_11"] = dates.iloc[:,i-10]
    one_day["Day_12"] = dates.iloc[:,i-11]
    one_day["Day_13"] = dates.iloc[:,i-12]
    one_day["Day_14"] = dates.iloc[:,i-13]
    one_day["Date"] = cols[i]
    nc_time = nc_counties.join(one_day)
    nc_time_series = nc_time_series.append(nc_time)
#    county_time.append(one_day)
#    print(one_day)
nc_time_series.head()


,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Yesterday,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date
1895,37001.0,Alamance,North Carolina,36.043470,-79.399761,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
1896,37003.0,Alexander,North Carolina,35.922380,-81.177519,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
1897,37005.0,Alleghany,North Carolina,36.493609,-81.128570,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
1898,37007.0,Anson,North Carolina,34.974032,-80.099533,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20
1899,37009.0,Ashe,North Carolina,36.432962,-81.498627,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3/18/20


In [38]:
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)

In [39]:
nc_time_series["Date"] = pd.to_datetime(nc_time_series.Date)
nc_time_series.dtypes

FIPS                     float64
Admin2                    object
Province_State            object
Lat                      float64
Long_                    float64
Confirmed                 object
Yesterday                 object
Day_3                     object
Day_4                     object
Day_5                     object
Day_6                     object
Day_7                     object
Day_8                     object
Day_9                     object
Day_10                    object
Day_11                    object
Day_12                    object
Day_13                    object
Day_14                    object
Date              datetime64[ns]
dtype: object

In [40]:
#nc_time_series["Date"] = nc_time_series["Date"].dt.to_period("D")

In [41]:
nc_time_series.tail()

,FIPS,Admin2,Province_State,Lat,Long_,Confirmed,Yesterday,Day_3,Day_4,Day_5,Day_6,Day_7,Day_8,Day_9,Day_10,Day_11,Day_12,Day_13,Day_14,Date
5895,37191.0,Wayne,North Carolina,35.364381,-78.000273,801,775,760,755,752,743,726,715,699,688,688,685,677,674,2020-05-15
5896,37193.0,Wilkes,North Carolina,36.208413,-81.162329,287,277,262,243,242,239,202,198,186,147,128,115,92,59,2020-05-15
5897,37195.0,Wilson,North Carolina,35.706255,-77.917978,222,221,217,216,216,212,203,199,192,192,182,180,175,173,2020-05-15
5898,37197.0,Yadkin,North Carolina,36.162226,-80.662394,92,80,79,60,54,54,49,39,33,27,25,24,23,23,2020-05-15
5899,37199.0,Yancey,North Carolina,35.900792,-82.312372,8,8,9,9,9,9,8,7,7,1,1,1,1,0,2020-05-15


In [42]:
#create and calculate a column for the number of new cases each day
nc_time_series["New_Cases"] = nc_time_series.apply(lambda row: row.Confirmed - row.Yesterday, axis=1)

In [43]:
#create and calculate a column with a rolling average
nc_time_series["Rolling_Avg"] = nc_time_series.apply(
    lambda row: (row.Confirmed + row.Yesterday + row.Day_3+row.Day_4
                 +row.Day_5+row.Day_6+row.Day_7+row.Day_8+row.Day_9+
                 row.Day_10+row.Day_11+row.Day_12+row.Day_13+row.Day_14)/14, axis = 1)

In [44]:
#now that Rolling_Avg is calculated, get rid of the unneeded calculation columns
nc_time_series = nc_time_series[["FIPS", "Admin2", "Province_State", "Confirmed", 
                                         "New_Cases", "Rolling_Avg", "Date"]]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
5895,37191.0,Wayne,North Carolina,801,26,724.142857,2020-05-15
5896,37193.0,Wilkes,North Carolina,287,10,191.214286,2020-05-15
5897,37195.0,Wilson,North Carolina,222,1,200.000000,2020-05-15
5898,37197.0,Yadkin,North Carolina,92,12,47.285714,2020-05-15
5899,37199.0,Yancey,North Carolina,8,0,5.571429,2020-05-15


In [45]:
nc_time_series.sort_values(["Admin2", "Date"], inplace = True)
nc_time_series.reset_index(inplace = True)
nc_time_series.drop("index", axis = 1, inplace = True)
nc_time_series.head(5)

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
0,37001.0,Alamance,North Carolina,0,0,0.000000,2020-03-18
1,37001.0,Alamance,North Carolina,0,0,0.000000,2020-03-19
2,37001.0,Alamance,North Carolina,1,1,0.071429,2020-03-20
3,37001.0,Alamance,North Carolina,2,1,0.214286,2020-03-21
4,37001.0,Alamance,North Carolina,2,0,0.357143,2020-03-22


In [46]:
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date
5895,37199.0,Yancey,North Carolina,9,0,3.142857,2020-05-11
5896,37199.0,Yancey,North Carolina,9,0,3.785714,2020-05-12
5897,37199.0,Yancey,North Carolina,9,0,4.428571,2020-05-13
5898,37199.0,Yancey,North Carolina,8,-1,5.000000,2020-05-14
5899,37199.0,Yancey,North Carolina,8,0,5.571429,2020-05-15


In [47]:
#add a column to calculate rolling average of new cases
nc_time_series["Temp"] = nc_time_series.Rolling_Avg.shift(1)
nc_time_series.head()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp
0,37001.0,Alamance,North Carolina,0,0,0.000000,2020-03-18,NaN
1,37001.0,Alamance,North Carolina,0,0,0.000000,2020-03-19,0.000000
2,37001.0,Alamance,North Carolina,1,1,0.071429,2020-03-20,0.000000
3,37001.0,Alamance,North Carolina,2,1,0.214286,2020-03-21,0.071429
4,37001.0,Alamance,North Carolina,2,0,0.357143,2020-03-22,0.214286


In [48]:
#create and calculate row with rolling average of new cases
nc_time_series["New_Case_Roll_Avg"] = nc_time_series.apply(lambda row: row.Rolling_Avg - row.Temp, axis=1)
nc_time_series = nc_time_series[nc_time_series["Date"] != "2020-03-18"]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,New_Cases,Rolling_Avg,Date,Temp,New_Case_Roll_Avg
5895,37199.0,Yancey,North Carolina,9,0,3.142857,2020-05-11,2.500000,0.642857
5896,37199.0,Yancey,North Carolina,9,0,3.785714,2020-05-12,3.142857,0.642857
5897,37199.0,Yancey,North Carolina,9,0,4.428571,2020-05-13,3.785714,0.642857
5898,37199.0,Yancey,North Carolina,8,-1,5.000000,2020-05-14,4.428571,0.571429
5899,37199.0,Yancey,North Carolina,8,0,5.571429,2020-05-15,5.000000,0.571429


In [49]:
#remove Temp column
nc_time_series = nc_time_series[["FIPS","Admin2","Province_State","Confirmed","Rolling_Avg",
                                       "New_Cases","New_Case_Roll_Avg","Date"]]
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date
5895,37199.0,Yancey,North Carolina,9,3.142857,0,0.642857,2020-05-11
5896,37199.0,Yancey,North Carolina,9,3.785714,0,0.642857,2020-05-12
5897,37199.0,Yancey,North Carolina,9,4.428571,0,0.642857,2020-05-13
5898,37199.0,Yancey,North Carolina,8,5.000000,-1,0.571429,2020-05-14
5899,37199.0,Yancey,North Carolina,8,5.571429,0,0.571429,2020-05-15


In [50]:
#add a column with a unique value that can be used to establish a relationship with other datasheets
nc_time_series["Unique"] = nc_time_series["Admin2"]+", "+nc_time_series["Date"].astype(str)
nc_time_series.tail()

,FIPS,Admin2,Province_State,Confirmed,Rolling_Avg,New_Cases,New_Case_Roll_Avg,Date,Unique
5895,37199.0,Yancey,North Carolina,9,3.142857,0,0.642857,2020-05-11,"Yancey, 2020-05-11"
5896,37199.0,Yancey,North Carolina,9,3.785714,0,0.642857,2020-05-12,"Yancey, 2020-05-12"
5897,37199.0,Yancey,North Carolina,9,4.428571,0,0.642857,2020-05-13,"Yancey, 2020-05-13"
5898,37199.0,Yancey,North Carolina,8,5.000000,-1,0.571429,2020-05-14,"Yancey, 2020-05-14"
5899,37199.0,Yancey,North Carolina,8,5.571429,0,0.571429,2020-05-15,"Yancey, 2020-05-15"


In [51]:
nc_time_series_change = nc_time_series[["FIPS","Admin2","Province_State","Rolling_Avg","New_Case_Roll_Avg","Date","Unique"]]

In [52]:
#add a temporary column to calculate the change in the 14-day rolling average of new cases
timeframe = 14

nc_time_series_change["Temp2"] = nc_time_series_change.New_Case_Roll_Avg.shift(timeframe)
nc_time_series_change.tail(timeframe)

C:\Users\becbo\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Temp2
5886,37199.0,Yancey,North Carolina,0.000000,0.000000,2020-05-02,"Yancey, 2020-05-02",0.0
5887,37199.0,Yancey,North Carolina,0.071429,0.071429,2020-05-03,"Yancey, 2020-05-03",0.0
5888,37199.0,Yancey,North Carolina,0.142857,0.071429,2020-05-04,"Yancey, 2020-05-04",0.0
5889,37199.0,Yancey,North Carolina,0.214286,0.071429,2020-05-05,"Yancey, 2020-05-05",0.0
5890,37199.0,Yancey,North Carolina,0.285714,0.071429,2020-05-06,"Yancey, 2020-05-06",0.0
5891,37199.0,Yancey,North Carolina,0.785714,0.500000,2020-05-07,"Yancey, 2020-05-07",0.0
5892,37199.0,Yancey,North Carolina,1.285714,0.500000,2020-05-08,"Yancey, 2020-05-08",0.0
5893,37199.0,Yancey,North Carolina,1.857143,0.571429,2020-05-09,"Yancey, 2020-05-09",0.0
5894,37199.0,Yancey,North Carolina,2.500000,0.642857,2020-05-10,"Yancey, 2020-05-10",0.0
5895,37199.0,Yancey,North Carolina,3.142857,0.642857,2020-05-11,"Yancey, 2020-05-11",0.0


In [53]:
#create Change column showing the change in the 14 day rolling average over the past 14 days
nc_time_series_change["Change"]= nc_time_series_change.apply(lambda row: (row.New_Case_Roll_Avg - row.Temp2)/timeframe, axis = 1)
nc_time_series_change.tail()                                                            

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Temp2,Change
5895,37199.0,Yancey,North Carolina,3.142857,0.642857,2020-05-11,"Yancey, 2020-05-11",0.0,0.045918
5896,37199.0,Yancey,North Carolina,3.785714,0.642857,2020-05-12,"Yancey, 2020-05-12",0.0,0.045918
5897,37199.0,Yancey,North Carolina,4.428571,0.642857,2020-05-13,"Yancey, 2020-05-13",0.0,0.045918
5898,37199.0,Yancey,North Carolina,5.000000,0.571429,2020-05-14,"Yancey, 2020-05-14",0.0,0.040816
5899,37199.0,Yancey,North Carolina,5.571429,0.571429,2020-05-15,"Yancey, 2020-05-15",0.0,0.040816


In [54]:
offset2 = datetime.timedelta(days = 2)
olddate = today - offset2
olddate

datetime.datetime(2020, 5, 14, 22, 30, 35, 281924)

In [55]:
nc_time_series_change = nc_time_series_change[nc_time_series_change["Date"] > olddate]
nc_time_series_change = nc_time_series_change[["FIPS","Admin2","Province_State",
                                               "Rolling_Avg","New_Case_Roll_Avg","Date","Unique","Change"]]
nc_time_series_change.tail()

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Change
5663,37191.0,Wayne,North Carolina,724.142857,10.214286,2020-05-15,"Wayne, 2020-05-15",-1.372449
5722,37193.0,Wilkes,North Carolina,191.214286,17.500000,2020-05-15,"Wilkes, 2020-05-15",1.056122
5781,37195.0,Wilson,North Carolina,200.000000,4.071429,2020-05-15,"Wilson, 2020-05-15",-0.061224
5840,37197.0,Yadkin,North Carolina,47.285714,5.357143,2020-05-15,"Yadkin, 2020-05-15",0.352041
5899,37199.0,Yancey,North Carolina,5.571429,0.571429,2020-05-15,"Yancey, 2020-05-15",0.040816


In [56]:
nc_time_series_change.reset_index(inplace = True)
nc_time_series_change.drop("index", axis = 1, inplace = True)
nc_time_series_change.head(5)

,FIPS,Admin2,Province_State,Rolling_Avg,New_Case_Roll_Avg,Date,Unique,Change
0,37001.0,Alamance,North Carolina,156.714286,6.214286,2020-05-15,"Alamance, 2020-05-15",0.127551
1,37003.0,Alexander,North Carolina,9.285714,0.785714,2020-05-15,"Alexander, 2020-05-15",0.051020
2,37005.0,Alleghany,North Carolina,7.571429,0.428571,2020-05-15,"Alleghany, 2020-05-15",0.020408
3,37007.0,Anson,North Carolina,35.500000,1.142857,2020-05-15,"Anson, 2020-05-15",0.020408
4,37009.0,Ashe,North Carolina,9.714286,0.857143,2020-05-15,"Ashe, 2020-05-15",0.056122


In [57]:
nc_time_series["Date"] = nc_time_series["Date"].dt.to_period("D")
nc_time_series_change["Date"] = nc_time_series_change["Date"].dt.to_period("D")

In [58]:
nc_time_series.to_excel("../clean_data/nc_rolling_avg.xlsx")
nc_time_series_change.to_excel("../clean_data/nc_rolling_avg_change.xlsx")